In [53]:
from __future__ import print_function
import mdtraj as md
import os
%matplotlib inline
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from contact_map import ContactMap, ContactFrequency, ContactDifference, ContactTrajectory
from concurrent.futures import ThreadPoolExecutor
import logging
import time

In [54]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
now_times = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [55]:
ref_path= '/mnt/nas1/lanwei-125/FGF5/FGF5-pos/FGFR1/'
peptdie_MD_paths = '/mnt/nas1/lanwei-125/FGF5/disulfide/MD/v1/'


In [56]:
def read_chain_residues(file_path):
    pdb_path = file_path/ (file_path.stem+'.pdb')
    with open(pdb_path) as f:
        lines = f.readlines()
        
    chains = {}
    for line in lines:
        if line.startswith('ATOM'):
            chain_id = line[21]
            residue_id = line[22:26].strip()
            unique_key = (chain_id, residue_id)

            if unique_key not in chains:
                chains[unique_key] = True

    residue_counts = {}
    for key in chains:
        chain_id = key[0]
        if chain_id not in residue_counts:
            residue_counts[chain_id] = 0
        residue_counts[chain_id] += 1
    residue_range = []
    for chain_id, residue_count in residue_counts.items():
        residue_range.append(residue_count)

    return residue_range


def read_pdb_trajectory(pdb, topology, trajectory):
    top = md.load_prmtop(topology)
    traj = md.load(trajectory,stride=500, top=pdb)
    return top, traj

def select_residues(prmtop, residue_range):
    receptor = prmtop.select(
        f"resid 0 to {residue_range[0]-1} and element !=H"
        )
    ligand = prmtop.select(
        f"resid {residue_range[0]} to {residue_range[0]+residue_range[1]-1} and element !=H"
        )
    logger.info(f"{now_times} receptor resid 0 to {residue_range[0]-1} and element !=H")
    logger.info(f"{now_times} ligand resid {residue_range[0]} to {residue_range[0]+residue_range[1]-1} and element !=H")
    
    return receptor, ligand

def make_contact_frequency(traj, query, cutoff=0.45, select_side = True):
    trajectory_contacts = ContactTrajectory(traj,query= query,cutoff=cutoff)
    freq = trajectory_contacts.contact_frequency()
    #freq.residue_contacts.most_common_idx()
    contact_lists = []

    for i in range(len(freq.residue_contacts.most_common_idx())):
        tup = freq.residue_contacts.most_common_idx()[i]
        if tup[1] >0.5:
            print(list(tup[0]),tup[1])
            if select_side:
                contact_lists.append(list(tup[0])[0])
            else :
                contact_lists.append(list(tup[0])[1])
    #contact_lists.sort()
    return set(contact_lists)

def data_path(flie_path):
    '''return pdb_path, top, traj'''
    top = flie_path / 'fixed.prmtop'
    traj = flie_path / 'fixed.mdcrd'
    pdb_path = flie_path / (flie_path.stem+'.pdb')
    return pdb_path, top, traj

def calc_contact_frequency(pdb_path, topology, trajectory):
    "returns a set of residues that are in contact with the ligand"
    residue_range =read_chain_residues(pdb_path) 
    top, traj = read_pdb_trajectory(pdb_path, topology, trajectory)
    _, ligand = select_residues(top, residue_range)
    contact_residue_list = make_contact_frequency(traj, ligand)
    return contact_residue_list

'''
def calc_contact_rate(ref_path, peptdie_MD_paths):
    ref_path = Path(ref_path)
    ref_top,  ref_traj,  ref_pdb_path = data_path(ref_path)
    ref_contact_residue_list = calc_contact_frequency(ref_top,  ref_traj,  ref_pdb_path )
    rates = []
    peptdie_MD_paths = Path(peptdie_MD_paths)
    for flie_path in peptdie_MD_paths.iterdir():
        if flie_path.is_dir():
            pdb_path, top, traj = data_path(flie_path)
            if top and traj and pdb_path is not None:
                contact_residue_list = calc_contact_frequency(pdb_path, top, traj )
                rate = len(contact_residue_list.intersection(ref_contact_residue_list))/len(ref_contact_residue_list)
                a = flie_path.stem, rate
                rates.append(a)
    return rates
'''

'\ndef calc_contact_rate(ref_path, peptdie_MD_paths):\n    ref_path = Path(ref_path)\n    ref_top,  ref_traj,  ref_pdb_path = data_path(ref_path)\n    ref_contact_residue_list = calc_contact_frequency(ref_top,  ref_traj,  ref_pdb_path )\n    rates = []\n    peptdie_MD_paths = Path(peptdie_MD_paths)\n    for flie_path in peptdie_MD_paths.iterdir():\n        if flie_path.is_dir():\n            pdb_path, top, traj = data_path(flie_path)\n            if top and traj and pdb_path is not None:\n                contact_residue_list = calc_contact_frequency(pdb_path, top, traj )\n                rate = len(contact_residue_list.intersection(ref_contact_residue_list))/len(ref_contact_residue_list)\n                a = flie_path.stem, rate\n                rates.append(a)\n    return rates\n'

In [57]:
ref_path = Path(ref_path)
peptdie_MD_paths = Path(peptdie_MD_paths)

ref_pdb_path, ref_top, ref_traj  = data_path(ref_path)
residue_range =read_chain_residues(ref_path) 
top, traj = read_pdb_trajectory(ref_pdb_path, ref_top, ref_traj)
receptor, ligand = select_residues(top, residue_range)
logger.info(f'receptor: {ligand}')
ref_contact_residue_list = make_contact_frequency(traj, ligand, select_side=False)
ref_contact_residue_list

2024-01-09 16:33:08,455 - INFO - 2024-01-09 16:27:43 receptor resid 0 to 224 and element !=H
2024-01-09 16:33:08,455 - INFO - 2024-01-09 16:27:43 receptor resid 0 to 224 and element !=H
2024-01-09 16:33:08,458 - INFO - 2024-01-09 16:27:43 ligand resid 225 to 355 and element !=H
2024-01-09 16:33:08,458 - INFO - 2024-01-09 16:27:43 ligand resid 225 to 355 and element !=H
2024-01-09 16:33:08,460 - INFO - receptor: [3533 3537 3539 ... 5621 5622 5623]
2024-01-09 16:33:08,460 - INFO - receptor: [3533 3537 3539 ... 5621 5622 5623]


[236, 230] 1.0
[298, 263] 1.0
[328, 335] 1.0
[347, 350] 1.0
[267, 262] 1.0
[352, 229] 1.0
[297, 279] 1.0
[280, 288] 1.0
[247, 239] 1.0
[274, 254] 1.0
[352, 311] 1.0
[345, 351] 1.0
[281, 326] 1.0
[336, 329] 1.0
[145, 263] 1.0
[313, 325] 1.0
[298, 295] 1.0
[261, 271] 1.0
[348, 351] 1.0
[231, 351] 1.0
[237, 229] 1.0
[312, 299] 1.0
[238, 247] 1.0
[297, 294] 1.0
[322, 318] 1.0
[241, 245] 1.0
[298, 314] 1.0
[350, 326] 1.0
[329, 349] 1.0
[280, 297] 1.0
[288, 279] 1.0
[297, 269] 1.0
[336, 330] 1.0
[251, 237] 1.0
[333, 247] 1.0
[299, 279] 1.0
[328, 349] 1.0
[312, 324] 1.0
[272, 277] 1.0
[248, 237] 1.0
[241, 247] 1.0
[330, 334] 1.0
[308, 28] 1.0
[348, 231] 1.0
[325, 285] 1.0
[312, 300] 1.0
[232, 331] 1.0
[256, 239] 1.0
[240, 245] 1.0
[272, 276] 1.0
[250, 236] 1.0
[228, 238] 1.0
[352, 329] 1.0
[283, 325] 1.0
[252, 237] 1.0
[282, 286] 1.0
[232, 348] 1.0
[352, 228] 1.0
[281, 325] 1.0
[260, 270] 1.0
[270, 278] 1.0
[238, 246] 1.0
[240, 279] 1.0
[289, 278] 1.0
[268, 261] 1.0
[282, 325] 1.0
[326, 311] 

{21,
 22,
 26,
 27,
 28,
 31,
 107,
 109,
 142,
 143,
 145,
 174,
 175,
 203,
 204,
 205,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 244,
 245,
 246,
 247,
 251,
 252,
 253,
 254,
 255,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 281,
 282,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 307,
 308,
 309,
 310,
 311,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 324,
 325,
 326,
 327,
 329,
 330,
 331,
 333,
 334,
 335,
 336,
 338,
 340,
 341,
 342,
 343,
 345,
 347,
 348,
 349,
 350,
 351,
 355}

In [ ]:
receptor, ligand = select_residues(top, residue_range)
logger.info(f'receptor: {ligand}')
ref_contact_residue_list = make_contact_frequency(traj, ligand)

In [ ]:
def calc_contact_rate(ref_path, peptdie_MD_paths):
    ref_path = Path(ref_path)
    peptdie_MD_paths = Path(peptdie_MD_paths)

    ref_pdb_path, ref_top, ref_traj  = data_path(ref_path)
    residue_range =read_chain_residues(ref_path) 
    top, traj = read_pdb_trajectory(ref_pdb_path, ref_top, ref_traj)
    receptor, _ = select_residues(top, residue_range)
    ref_contact_residue_list = make_contact_frequency(traj, receptor)

    rates = []
    def process_fly(flies_path):
        pdb_path, top, traj = data_path(flies_path)
        if top and traj and pdb_path is not None:
            contact_residue_list = calc_contact_frequency(pdb_path, top, traj)
            rate = len(contact_residue_list.intersection(ref_contact_residue_list)) / len(ref_contact_residue_list)
            return flies_path, rate
        return None

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_fly, flies_path) for flies_path in peptdie_MD_paths.iterdir() if flies_path.is_dir()]

        for future in futures:
            result = future.result()
            if result:
                rates.append(result)

    return rates

In [ ]:
rates = calc_contact_rate(ref_path, peptdie_MD_paths)